In [23]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Sets
processes = ["P1", "P2"]
system_time = list(range(2020, 2030))
process_time = [0, 1, 2, 3]
functional_flow = "F1"
intermediate_flows = ["I1", "I2"]
elementary_flows = ["CO2", "CH4"]

# Decision variables: how many units of each process to start at a given system time
model.x = pyo.Var(processes, system_time, domain=pyo.NonNegativeReals)

# Demand (known)
demand = {
    2020: 0, 2021: 0, 2022: 10, 2023: 10, 2024: 10,
    2025: 10, 2026: 10, 2027: 10, 2028: 10, 2029: 10
}

# Fixed parameters (sparse form)
foreground_production = {
    ("P1", 1): 0.5, ("P1", 2): 0.5,
    ("P2", 1): 0.5, ("P2", 2): 0.5,
}

foreground_technosphere = {
    ("P1", "I1", 0): 1000,
    ("P2", "I2", 0): 1, 
}

foreground_biosphere = {

}

background_inventory = {
    ("db_2020", "I1", "CO2"): 1, ("db_2020", "I2", "CH4"): 1,
    ("db_2030", "I1", "CO2"): 1, ("db_2030", "I2", "CH4"): 1,
}

# Mapping: The proportion of each database used per year
mapping = {}
for y in system_time:
    mapping[("db_2020", y)] = 1 - (y - 2020) * 0.1
    mapping[("db_2030", y)] = 1 - mapping[("db_2020", y)]

# Characterization factor (impact per unit emission)
characterization = {
    "CO2": {
        2020: 8.856378067710995e-14,
        2021: 8.78632948322376e-14,
        2022: 8.716115201983699e-14,
        2023: 8.645732530491629e-14,
        2024: 8.575178705349817e-14,
        2025: 8.50445089133486e-14,
        2026: 8.433546179417147e-14,
        2027: 8.362461584725384e-14,
        2028: 8.291194044454685e-14,
        2029: 8.219740415716608e-14,
    },
    "CH4": {
        2020: 2.3651673669270527e-12,
        2021: 2.3651198384711042e-12,
        2022: 2.3650681065838066e-12,
        2023: 2.36501179951239e-12,
        2024: 2.3649505126261558e-12,
        2025: 2.3648838055087402e-12,
        2026: 2.364811198793221e-12,
        2027: 2.3647321707173162e-12,
        2028: 2.3646461533739266e-12,
        2029: 2.364552528630073e-12,
    }
}

# Constraint: meet demand each year
def demand_constraint_rule(model, y):
    expr = 0
    active = False  # Flag to see if any vars are included
    for p in processes:
        for pt in process_time:
            st = y - pt
            if st in system_time and (p, pt) in foreground_production:
                active = True
                expr += model.x[p, st] * foreground_production[p, pt]
    if not active:
        return pyo.Constraint.Feasible  # no variables involved → skip
    return expr >= demand[y]



model.demand_constraint = pyo.Constraint(system_time, rule=demand_constraint_rule)

# Objective: minimize total environmental impact
def impact_expression(model):
    impact = 0
    for p in processes:
        for st in system_time:
            for pt in process_time:
                y = st + pt
                if y not in system_time:
                    continue

                # Background emissions from intermediates
                for i in intermediate_flows:
                    amount = foreground_technosphere.get((p, i, pt), 0)
                    for e in elementary_flows:
                        for db in ["db_2020", "db_2030"]:
                            mix = mapping.get((db, y), 0)
                            bg_emiss = background_inventory.get((db, i, e), 0)
                            cf = characterization[e][y]
                            impact += model.x[p, st] * amount * bg_emiss * mix * cf

                # Foreground direct emissions
                for e in elementary_flows:
                    fg_emiss = foreground_biosphere.get((p, e, pt), 0)
                    cf = characterization[e][y]
                    impact += model.x[p, st] * fg_emiss * cf

    return impact

model.objective = pyo.Objective(rule=impact_expression, sense=pyo.minimize)

# Solve
solver = pyo.SolverFactory("gurobi")
results = solver.solve(model)

# Output
for p in processes:
    for y in system_time:
        val = pyo.value(model.x[p, y])
        if val > 0.001:
            print(f"{p} starts in {y}: {val:.2f} units")

print(f"Total environmental impact: {pyo.value(model.objective):.3e}")



P1 starts in 2021: 20.00 units
P1 starts in 2023: 20.00 units
P1 starts in 2025: 20.00 units
P1 starts in 2027: 20.00 units
Total environmental impact: 6.860e-09
